# Aggregation and Grouping

In [118]:
import numpy as np
import pandas as pd

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [119]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [120]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


## the usage of ``.describe()``

In [121]:
planets.describe()

,number,orbital_period,mass,distance,year
count,1035.000000,992.000000,513.000000,808.000000,1035.000000
mean,1.785507,2002.917596,2.638161,264.069282,2009.070531
std,1.240976,26014.728304,3.818617,733.116493,3.972567
min,1.000000,0.090706,0.003600,1.350000,1989.000000
25%,1.000000,5.442540,0.229000,32.560000,2007.000000
50%,1.000000,39.979500,1.260000,55.250000,2010.000000
75%,2.000000,526.005000,3.040000,178.500000,2012.000000
max,7.000000,730000.000000,25.000000,8500.000000,2014.000000


In [122]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


The following table summarizes some other built-in Pandas aggregations:

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |

These are all methods of ``DataFrame`` and ``Series`` objects.

## GroupBy: Split, Apply, Combine

### Split, apply, combine

A canonical example of this split-apply-combine operation, where the "apply" is a summation aggregation, is illustrated in this figure:

![](figures/03.08-split-apply-combine.png)
[figure source in Appendix](06.00-Figure-Code.ipynb#Split-Apply-Combine)

This makes clear what the ``groupby`` accomplishes:

- The *split* step involves breaking up and grouping a ``DataFrame`` depending on the value of the specified key.
- The *apply* step involves computing some function, usually an aggregate, transformation, or filtering, within the individual groups.
- The *combine* step merges the results of these operations into an output array.

While this could certainly be done manually using some combination of the masking, aggregation, and merging commands covered earlier, an important realization is that *the intermediate splits do not need to be explicitly instantiated*. Rather, the ``GroupBy`` can (often) do this in a single pass over the data, updating the sum, mean, count, min, or other aggregate for each group along the way.
The power of the ``GroupBy`` is that it abstracts away these steps: the user need not think about *how* the computation is done under the hood, but rather thinks about the *operation as a whole*.

As a concrete example, let's take a look at using Pandas for the computation shown in this diagram.
We'll start by creating the input ``DataFrame``:

# The GroupBy object

## Column indexing

In [123]:
planets.groupby('method')

In [124]:
planets.groupby('method')['orbital_period']

In [125]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

In [126]:
planets.groupby('method').sum()

,number,orbital_period,mass,distance,year
method,,,,,
Astrometry,2,1.262360e+03,0.00000,35.75,4023
Eclipse Timing Variations,15,4.276480e+04,10.25000,1261.44,18090
Imaging,50,1.418973e+06,0.00000,2166.91,76347
Microlensing,27,2.207500e+04,0.00000,41440.00,46225
Orbital Brightness Modulation,5,2.127920e+00,0.00000,2360.00,6035
Pulsar Timing,11,3.671511e+04,0.00000,1200.00,9992
Pulsation Timing Variations,1,1.170000e+03,0.00000,0.00,2007
Radial Velocity,952,4.553151e+05,1341.65638,27348.11,1110158
Transit,776,8.377523e+03,1.47000,134242.77,798461


## Iteration over groups

The ``GroupBy`` object supports direct iteration over the groups, returning each group as a ``Series`` or ``DataFrame``:

In [127]:
for (method, group) in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


## Dispatch methods

In [128]:
planets.groupby('method')['year'].describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


# Aggregate, filter, transform, apply

## Aggregation

We're now familiar with ``GroupBy`` aggregations with ``sum()``, ``median()``, and the like, but the ``aggregate()`` method allows for even more flexibility.
It can take a string, a function, or a list thereof, and compute all the aggregates at once.

In [129]:
planets.groupby('method').aggregate(['min', np.median, max]).stack()

number  orbital_period     mass  \
method                                                                  
Astrometry                    min          1      246.360000      NaN   
                              median       1      631.180000      NaN   
                              max          1     1016.000000      NaN   
Eclipse Timing Variations     min          1     1916.250000   4.2000   
                              median       2     4343.500000   5.1250   
                              max          2    10220.000000   6.0500   
Imaging                       min          1     4639.150000      NaN   
                              median       1    27500.000000      NaN   
                              max          4   730000.000000      NaN   
Microlensing                  min          1     1825.000000      NaN   
                              median       1     3300.000000      NaN   
                              max          2     5100.000000      NaN   
Orbital Brightness Modulation min          1        0.240104      NaN   
                              median       2        0.342887      NaN   
                              max          2        1.544929      NaN   
Pulsar Timing                 min          1        0.090706      NaN   
                              median       3       66.541900      NaN   
                              max          3    36525.000000      NaN   
Pulsation Timing Variations   min          1     1170.000000      NaN   
                              median       1     1170.000000      NaN   
                              max          1     1170.000000      NaN   
Radial Velocity               min          1        0.736540   0.0036   
                              median       1      360.200000   1.2600   
                              max          6    17337.500000  25.0000   
Transit                       min          1        0.355000   1.4700   
                              median       1        5.714932   1.4700   
                              max          7      331.600590   1.4700   
Transit Timing Variations     min          2       22.339500      NaN   
                              median       2       57.011000      NaN   
                              max          3      160.000000      NaN   

                                      distance    year  
method                                                  
Astrometry                    min       14.980  2010.0  
                              median    17.875  2011.5  
                              max       20.770  2013.0  
Eclipse Timing Variations     min      130.720  2008.0  
                              median   315.360  2010.0  
                              max      500.000  2012.0  
Imaging                       min        7.690  2004.0  
                              median    40.395  2009.0  
                              max      165.000  2013.0  
Microlensing                  min     1760.000  2004.0  
                              median  3840.000  2010.0  
                              max     7720.000  2013.0  
Orbital Brightness Modulation min     1180.000  2011.0  
                              median  1180.000  2011.0  
                              max     1180.000  2013.0  
Pulsar Timing                 min     1200.000  1992.0  
                              median  1200.000  1994.0  
                              max     1200.000  2011.0  
Pulsation Timing Variations   min          NaN  2007.0  
                              median       NaN  2007.0  
                              max          NaN  2007.0  
Radial Velocity               min        1.350  1989.0  
                              median    40.445  2009.0  
                              max      354.000  2014.0  
Transit                       min       38.000  2002.0  
                              median   341.000  2012.0  
                              max     8500.000  2014.0  
Transit Timing Variations     min      339.000  

In [130]:
# pass a dictionary mapping column names to operations to be applied on that column
planets.groupby('method').aggregate({'mass': min,
                             'year': 'max'})

,mass,year
method,,
Astrometry,NaN,2013
Eclipse Timing Variations,4.2000,2012
Imaging,NaN,2013
Microlensing,NaN,2013
Orbital Brightness Modulation,NaN,2013
Pulsar Timing,NaN,2011
Pulsation Timing Variations,NaN,2007
Radial Velocity,0.0036,2014
Transit,1.4700,2014


## Filtering

A filtering operation allows you to drop data based on the group properties.

In [131]:
def filter_func(x):
    return x['year'].min() > 2000

display( "planets.head()", "planets.groupby('method').filter(filter_func).head()")
# the return value of filter is a dataframe

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009
,method,number,orbital_period,mass,distance,year
29,Imaging,1,NaN,NaN,45.52,2005
30,Imaging,1,NaN,NaN,165.00,2007
31,Imaging,1,NaN,NaN,140.00,2004
32,Eclipse Timing Variations,1,10220.0,6.05,NaN,2009


## Transformation

While aggregation must return a reduced version of the data, transformation can return some transformed version of the full data to recombine.

In [132]:
# A common example is to center the data by subtracting the group-wise mean:
planets.groupby('method').transform(lambda x: x - x.mean()).head()

,number,orbital_period,mass,distance,year
0,-0.721519,-554.05468,4.469301,25.799792,-1.518987
1,-0.721519,51.41932,-0.420699,5.349792,0.481013
2,-0.721519,-60.35468,-0.030699,-31.760208,3.481013
3,-0.721519,-497.32468,16.769301,59.019792,-0.518987
4,-0.721519,-307.13468,7.869301,67.869792,1.481013


## Apply

The ``apply()`` method lets you apply an arbitrary function to the group results.
The function should take a ``DataFrame``, and return either a Pandas object (e.g., ``DataFrame``, ``Series``) or a scalar; the combine operation will be tailored to the type of output returned.

In [133]:
def temp(x):
    # x is a DataFrame of group values
    x['mass'] /= x['mass'].sum()
    return x

planets.groupby('method').apply(temp).head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,0.005292,77.40,2006
1,Radial Velocity,1,874.774,0.001647,56.95,2008
2,Radial Velocity,1,763.000,0.001938,19.84,2011
3,Radial Velocity,1,326.030,0.014460,110.62,2007
4,Radial Velocity,1,516.220,0.007826,119.47,2009


## Specifying the split key

### A list, array, series, or index providing the grouping keys

The key can be any series or list with a length matching that of the ``DataFrame``

In [134]:
print(planets.shape[0])
L = []
for i in range(5):
    label = [i] * int(planets.shape[0]/5)
    L += label
planets.groupby(L).sum()

1035


,number,orbital_period,mass,distance,year
0,347,492682.392359,289.02360,31335.44,415774
1,355,138981.478252,453.85088,10671.68,415518
2,303,182525.227264,540.42940,11601.90,415517
3,537,333911.234373,47.99250,73574.03,416339
4,306,838793.923078,22.08000,86184.93,416240


In [135]:
planets.groupby(planets['method']).median()

,number,orbital_period,mass,distance,year
method,,,,,
Astrometry,1.0,631.180000,NaN,17.875,2011.5
Eclipse Timing Variations,2.0,4343.500000,5.125,315.360,2010.0
Imaging,1.0,27500.000000,NaN,40.395,2009.0
Microlensing,1.0,3300.000000,NaN,3840.000,2010.0
Orbital Brightness Modulation,2.0,0.342887,NaN,1180.000,2011.0
Pulsar Timing,3.0,66.541900,NaN,1200.000,1994.0
Pulsation Timing Variations,1.0,1170.000000,NaN,NaN,2007.0
Radial Velocity,1.0,360.200000,1.260,40.445,2009.0
Transit,1.0,5.714932,1.470,341.000,2012.0


### A dictionary or series mapping index to group

Another method is to provide a dictionary that maps index values to the group keys:

In [136]:
planets2 = planets.set_index('method')
mapping = {'Astrometry': '1', 'Eclipse Timing Variations': '3', 'Imaging': '2', 'Microlensing': '1', 'Orbital Brightness Modulation': '1'}
planets2.groupby(mapping).median()

,number,orbital_period,mass,distance,year
1,1,1897.5,NaN,2685.000,2011
2,1,27500.0,NaN,40.395,2009
3,2,4343.5,5.125,315.360,2010


### Any Python function

Similar to mapping, you can pass any Python function that will input the index value and output the group:

In [137]:
planets2.groupby(str.lower).median()

,number,orbital_period,mass,distance,year
astrometry,1.0,631.180000,NaN,17.875,2011.5
eclipse timing variations,2.0,4343.500000,5.125,315.360,2010.0
imaging,1.0,27500.000000,NaN,40.395,2009.0
microlensing,1.0,3300.000000,NaN,3840.000,2010.0
orbital brightness modulation,2.0,0.342887,NaN,1180.000,2011.0
pulsar timing,3.0,66.541900,NaN,1200.000,1994.0
pulsation timing variations,1.0,1170.000000,NaN,NaN,2007.0
radial velocity,1.0,360.200000,1.260,40.445,2009.0
transit,1.0,5.714932,1.470,341.000,2012.0
transit timing variations,2.0,57.011000,NaN,855.000,2012.5


### A list of valid keys

Further, any of the preceding key choices can be combined to group on a multi-index:

In [138]:
planets2.groupby([mapping, str.upper]).median()

number  orbital_period   mass  distance  \
1 ASTROMETRY                        1.0      631.180000    NaN    17.875   
  MICROLENSING                      1.0     3300.000000    NaN  3840.000   
  ORBITAL BRIGHTNESS MODULATION     2.0        0.342887    NaN  1180.000   
2 IMAGING                           1.0    27500.000000    NaN    40.395   
3 ECLIPSE TIMING VARIATIONS         2.0     4343.500000  5.125   315.360   

                                   year  
1 ASTROMETRY                     2011.5  
  MICROLENSING                   2010.0  
  ORBITAL BRIGHTNESS MODULATION  2011.0  
2 IMAGING                        2009.0  
3 ECLIPSE TIMING VARIATIONS      2010.0

### Grouping example:

In [139]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0
